In [3]:
import streamlit as st
from groq import Groq
import base64
import os
import time
import json

# Groq client initialization
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# In-memory storage for user data (for simplicity)
users_db = {}  # Dictionary to store user credentials (username: password)

# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function to get chat completion from Groq API
def get_chat_response(image_path, user_message):
    # Encode image to base64
    base64_image = encode_image(image_path)

    # Send request to Groq LLM API
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_message},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model="llama-3.2-11b-vision-preview",
    )

    return chat_completion.choices[0].message.content


# Streamlit app layout
def main():
    st.title("LLM Chatbot with Groq API")

    # Create session state for login, history, and user info
    if "logged_in" not in st.session_state:
        st.session_state.logged_in = False
    if "user_history" not in st.session_state:
        st.session_state.user_history = []

    # Login page
    if not st.session_state.logged_in:
        st.header("Login / Register")

        with st.form(key="login_form"):
            username = st.text_input("Username", key="username")
            password = st.text_input("Password", type="password", key="password")
            login_form_submit = st.form_submit_button("Login")
            register_form_submit = st.form_submit_button("Register")

            if login_form_submit:
                # Check if username exists and password matches
                if username in users_db and users_db[username] == password:
                    st.session_state.logged_in = True
                    st.session_state.username = username
                    st.success(f"Logged in as {st.session_state.username}")
                    time.sleep(1)
                    st.experimental_rerun()
                else:
                    st.error("Invalid username or password!")

            if register_form_submit:
                # Register new user
                if username in users_db:
                    st.error("Username already exists. Please choose a different one.")
                else:
                    users_db[username] = password
                    st.success(f"Registration successful! You can now log in as {username}.")
                    time.sleep(1)
                    st.experimental_rerun()
    else:
        # Sidebar options after login
        with st.sidebar:
            st.subheader("Welcome, " + st.session_state.username)
            option = st.selectbox("Select", ["Chat", "History", "Logout"])

            if option == "Chat":
                chat_interface()
            elif option == "History":
                show_history()
            elif option == "Logout":
                st.session_state.logged_in = False
                st.session_state.user_history = []
                st.success("Logged out successfully!")
                st.experimental_rerun()


# Chat interface
def chat_interface():
    st.header("Chat with LLM")

    # Upload image for LLM to analyze
    uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

    # User text input for chat
    user_message = st.text_input("Your message:")

    if uploaded_image and user_message:
        with st.spinner("Processing..."):
            # Save image to a temporary location
            image_path = "temp_image.jpg"
            with open(image_path, "wb") as f:
                f.write(uploaded_image.getbuffer())

            # Get LLM response
            response = get_chat_response(image_path, user_message)
            st.write("LLM's Response:", response)

            # Add chat to history
            st.session_state.user_history.append({"message": user_message, "response": response})

    # Display previous chats if any
    if st.session_state.user_history:
        st.subheader("Chat History")
        for chat in st.session_state.user_history:
            st.write(f"**You**: {chat['message']}")
            st.write(f"**LLM**: {chat['response']}")


# Show chat history
def show_history():
    st.header("Chat History")
    if st.session_state.user_history:
        for chat in st.session_state.user_history:
            st.write(f"**You**: {chat['message']}")
            st.write(f"**LLM**: {chat['response']}")
    else:
        st.write("No chat history found.")


# Run the app
if __name__ == "__main__":
    main()


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [2]:
pip install streamlit


  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 220.2 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 252.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 256.3 kB/s eta 0:00:0000:0100:02
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 300.4 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
